### 图像识别
<img src='images/cifar10.png' />
<br>
该数据集共有60000张32*32*3的彩色图像，分为10个类，每类6000张图。50000张用作训练，10000张用作测试。
<br>训练集中50000张图片，每个类别5000张。
<br>测试集中10000张图片，每个类别1000张。

In [ ]:
import keras
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from pathlib import Path

import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [ ]:
# 加载数据集
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# 数据集归一化(0-1之间)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [ ]:
# 标签转换为 One-Hot 编码
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

In [ ]:
# 定义CNN网络结构
model = Sequential()

model.add(Conv2D(32, (3, 3), padding='same', input_shape=(32, 32, 3), activation="relu"))
model.add(Conv2D(32, (3, 3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same', activation="relu"))
model.add(Conv2D(64, (3, 3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dropout(0.25))

model.add(Dense(512, activation="relu"))
model.add(Dropout(0.5))

model.add(Dense(10, activation="softmax"))

In [ ]:
# 编译CNN模型
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)
model.summary()

In [ ]:
# 训练CNN模型
history = model.fit(
    x_train,
    y_train,
    batch_size=64,
    epochs=3,
    validation_data=(x_test, y_test),
    shuffle=True
)

In [ ]:
# 模型在训练集和验证集上的预测准确率变化曲线
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.savefig("accuracy.png")
plt.show()

# 模型在训练集和验证集上的损失函数曲线
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.savefig("loss.png")
plt.show()

In [ ]:
# 保存CNN网络结构
model_structure = model.to_json()
f = Path("model_structure.json")
f.write_text(model_structure)

# 保存训练好的权重参数
model.save_weights("model_weights.h5")